In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0


In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-yrnvxw1i
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-yrnvxw1i
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4304 sha256=6324be187f1a12d823c7070ba5599ce442a72ab457969e1c1b70fb8be365369c
  Stored in directory: /tmp/pip-ephem-wheel-cache-u54ee6vn/wheels/f3/08/cc/e2b5b0e1c92df07dbb50a6f024a68ce090f5e7b2316b41756d
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
%%cu
#include <iostream>
using namespace std;
int main()
{
	cout << "Welcome To GeeksforGeeks"<<endl<<1;
	return 0;
}


Welcome To GeeksforGeeks
1


In [ ]:
%%cu
#include <iostream>
#include <cstdio>

using namespace std;
__global__ void add(int *a,int *b,int *c){
    *c=*a+*b;
}

int main(){
    int a,b,c; // host copies of a, b, c
    int *d_a,*d_b,*d_c; // device copies of a, b, c
    int size=sizeof(int);

    // Allocate space for device copies of a, b, c
    cudaMalloc((void **)&d_a,size);
    cudaMalloc((void **)&d_b,size);
    cudaMalloc((void **)&d_c,size);

    a=10,b=3;

    // Copy inputs to device
    cudaMemcpy(d_a,&a,size,cudaMemcpyHostToDevice);
    cudaMemcpy(d_b,&b,size,cudaMemcpyHostToDevice);

    // Launch add() kernel on GPU
    add<<<1,1>>>(d_a,d_b,d_c);

    // Copy result back to host
    cudaMemcpy(&c,d_c,size,cudaMemcpyDeviceToHost);

    cout<<c<<endl;

    //Free the space
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);

    return 0;
}

13



In [ ]:
%%cu
#include "stdio.h"
#include <iostream>

using namespace std;

// Defining number of elements in Array
#define N 10

__global__ void add(int *a,int *b,int *c){
    int i=blockIdx.x * blockDim.x + threadIdx.x;

    if(i<N){
        c[i]=a[i]+b[i];
    }
}

int main(){
    int h_a[N],h_b[N],h_c[N];

    int *d_a,*d_b,*d_c;

    cudaMalloc((void**)&d_a,N * sizeof(int));
    cudaMalloc((void**)&d_b,N * sizeof(int));
    cudaMalloc((void**)&d_c,N * sizeof(int));
    
    for (int i = 0; i < N; i++) {
        h_a[i] = 10;
        h_b[i] = 2 ;
    }

    cudaMemcpy(d_a,h_a,N * sizeof(int),cudaMemcpyHostToDevice);
    cudaMemcpy(d_b,h_b,N * sizeof(int),cudaMemcpyHostToDevice);

    add<<<N/2,2>>>(d_a,d_b,d_c);

    cudaMemcpy(h_c,d_c,N * sizeof(int),cudaMemcpyDeviceToHost);

    for (int i = 0; i < N; i++) {
        printf("%d\n",h_c[i]);
    }
    
    return 0;
}

12
12
12
12
12
12
12
12
12
12



## Addition of Matrix

In [ ]:
%%cu
#include <stdio.h>
#include <stdlib.h>

#define N 5
#define BLOCK_DIM 10

__global__ void matrixAdd (int *a, int *b, int *c) {
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    int row = blockIdx.y * blockDim.y + threadIdx.y;

    int index = col + row * N;

    if (col < N && row < N) {
        c[index] = a[index] + b[index];
    }

}

int main() {
    int a[N][N], b[N][N], c[N][N];
    int *dev_a, *dev_b, *dev_c;

    int size = N * N * sizeof(int);

    for(int i=0; i<N; i++)
        for (int j=0; j<N; j++){
            a[i][j] = 10;
            b[i][j] = 2;
        }

    cudaMalloc((void**)&dev_a, size);
    cudaMalloc((void**)&dev_b, size);
    cudaMalloc((void**)&dev_c, size);

    cudaMemcpy(dev_a, a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(dev_b, b, size, cudaMemcpyHostToDevice);

    dim3 dimBlock(BLOCK_DIM, BLOCK_DIM);
    //dim3 dimGrid((int)ceil(N/dimBlock.x),(int)ceil(N/dimBlock.y));
    dim3 dimGrid((N+dimBlock.x-1)/dimBlock.x, (N+dimBlock.y-1)/dimBlock.y);
    printf("dimGrid.x = %d, dimGrid.y = %d\n", dimGrid.x, dimGrid.y);
    matrixAdd<<<dimGrid,dimBlock>>>(dev_a,dev_b,dev_c);
    cudaMemcpy(c, dev_c, size, cudaMemcpyDeviceToHost);
    printf("Matrix a is:\n");
    for(int i=0; i<N; i++){
        for (int j=0; j<N; j++){
            printf("%d\t", a[i][j] );
        }
        printf("\n");
    }

    printf("Matrix b is:\n");
    for(int i=0; i<N; i++){
        for (int j=0; j<N; j++){
            printf("%d\t", b[i][j] );
        }
        printf("\n");
    }

    printf("Result after a*b is:\n");
    for(int i=0; i<N; i++){
        for (int j=0; j<N; j++){
            printf("%d\t", c[i][j] );
        }
        printf("\n");
    }


    cudaFree(dev_a);
    cudaFree(dev_b);
    cudaFree(dev_c);
}

dimGrid.x = 1, dimGrid.y = 1
Matrix a is:
10	10	10	10	10	
10	10	10	10	10	
10	10	10	10	10	
10	10	10	10	10	
10	10	10	10	10	
Matrix b is:
2	2	2	2	2	
2	2	2	2	2	
2	2	2	2	2	
2	2	2	2	2	
2	2	2	2	2	
Result after a*b is:
12	12	12	12	12	
12	12	12	12	12	
12	12	12	12	12	
12	12	12	12	12	
12	12	12	12	12	



## Multiplication of Matrix

In [ ]:
%%cu
#include <stdio.h>
#include <stdlib.h>

#define N 5
#define BLOCK_DIM 10

__global__ void matrixMult (int *a, int *b, int *c) {
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    int row = blockIdx.y * blockDim.y + threadIdx.y;

    

    if (col < N && row < N) {
        int tempSum=0;

        for(int i=0;i<N;i++){
            tempSum += a[row * N + i] * b[i * N + col];
        }

        c[row * N + col] = tempSum;
    }

}

int main() {
    int a[N][N], b[N][N], c[N][N];
    int *dev_a, *dev_b, *dev_c;

    int size = N * N * sizeof(int);

    for(int i=0; i<N; i++)
        for (int j=0; j<N; j++){
            a[i][j] = 3;
            b[i][j] = 2;
        }

    cudaMalloc((void**)&dev_a, size);
    cudaMalloc((void**)&dev_b, size);
    cudaMalloc((void**)&dev_c, size);

    cudaMemcpy(dev_a, a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(dev_b, b, size, cudaMemcpyHostToDevice);

    dim3 dimBlock(BLOCK_DIM, BLOCK_DIM);
    //dim3 dimGrid((int)ceil(N/dimBlock.x),(int)ceil(N/dimBlock.y));
    dim3 dimGrid((N+dimBlock.x-1)/dimBlock.x, (N+dimBlock.y-1)/dimBlock.y);
    printf("dimGrid.x = %d, dimGrid.y = %d\n", dimGrid.x, dimGrid.y);
    matrixMult<<<dimGrid,dimBlock>>>(dev_a,dev_b,dev_c);
    cudaMemcpy(c, dev_c, size, cudaMemcpyDeviceToHost);
    printf("Matrix x :\n");
    for(int i=0; i<N; i++){
        for (int j=0; j<N; j++){
            printf("%d\t", a[i][j] );
        }
        printf("\n");
    }

    printf("Matrix y :\n");
    for(int i=0; i<N; i++){
        for (int j=0; j<N; j++){
            printf("%d\t", b[i][j] );
        }
        printf("\n");
    }

    printf("Result after x*y :\n");
    for(int i=0; i<N; i++){
        for (int j=0; j<N; j++){
            printf("%d\t", c[i][j] );
        }
        printf("\n");
    }


    cudaFree(dev_a);
    cudaFree(dev_b);
    cudaFree(dev_c);
}

dimGrid.x = 1, dimGrid.y = 1
Matrix x :
3	3	3	3	3	
3	3	3	3	3	
3	3	3	3	3	
3	3	3	3	3	
3	3	3	3	3	
Matrix y :
2	2	2	2	2	
2	2	2	2	2	
2	2	2	2	2	
2	2	2	2	2	
2	2	2	2	2	
Result after x*y :
30	30	30	30	30	
30	30	30	30	30	
30	30	30	30	30	
30	30	30	30	30	
30	30	30	30	30	



In [ ]:
%%cu
#include <stdio.h>
#include <stdlib.h>

#define N 5
#define BLOCK_DIM 10

__global__ void matrixTrans(int *ain, int *aout) {
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    int row = blockIdx.y * blockDim.y + threadIdx.y;

    

    if (col < N && row < N) {
        aout[col*N+row]=ain[row*N+col];
    }

}

int main() {
    int a_in[N][N], a_out[N][N];;
    int *dev_ain, *dev_aout;

    int size = N * N * sizeof(int);

    for(int i=0; i<N; i++)
        for (int j=0; j<N; j++){
            a_in[i][j] = i*i+j+1;
        }

    cudaMalloc((void**)&dev_ain, size);
    cudaMalloc((void**)&dev_aout, size);

    cudaMemcpy(dev_ain, a_in, size, cudaMemcpyHostToDevice);
    cudaMemcpy(dev_aout, a_out, size, cudaMemcpyHostToDevice);

    dim3 dimBlock(BLOCK_DIM, BLOCK_DIM);
    //dim3 dimGrid((int)ceil(N/dimBlock.x),(int)ceil(N/dimBlock.y));
    dim3 dimGrid((N+dimBlock.x-1)/dimBlock.x, (N+dimBlock.y-1)/dimBlock.y);
    printf("dimGrid.x = %d, dimGrid.y = %d\n", dimGrid.x, dimGrid.y);
    matrixTrans<<<dimGrid,dimBlock>>>(dev_ain,dev_aout);
    cudaMemcpy(a_out, dev_aout, size, cudaMemcpyDeviceToHost);
    printf("Input Matrix is:\n");
    for(int i=0; i<N; i++){
        for (int j=0; j<N; j++){
            printf("%d\t", a_in[i][j] );
        }
        printf("\n");
    }

    printf("Transpose Matrix is:\n");
    for(int i=0; i<N; i++){
        for (int j=0; j<N; j++){
            printf("%d\t", a_out[i][j] );
        }
        printf("\n");
    }

}

dimGrid.x = 1, dimGrid.y = 1
Input Matrix is:
1	2	3	4	5	
2	3	4	5	6	
5	6	7	8	9	
10	11	12	13	14	
17	18	19	20	21	
Transpose Matrix is:
1	2	5	10	17	
2	3	6	11	18	
3	4	7	12	19	
4	5	8	13	20	
5	6	9	14	21	

